In [1]:
import os, sys, time
from time import sleep
from pathlib import Path
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import pyodbc
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows
import tempfile
from ttkthemes import ThemedStyle
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import messagebox, filedialog, commondialog

In [2]:
class TISS_Metrics(): # {
    
    user_name = str(os.getlogin())
    outbound_dir = "C:/data/outbound/" + str(pd.Timestamp.now())[:10]
    desktop_dir = "OneDrive - Agilent Technologies/Desktop"
    
    def __init__(self, the_logger, materials_file): # {
        self.the_logger = the_logger
        self.materials_file = materials_file
        # create dataframe from infile
        self.df_infile = pd.read_csv(self.materials_file, header=0, dtype=np.str, engine='python')
        # EXPORT?
        self.df_infile.to_csv(os.path.join(self.outbound_dir, "df_infile-"
                                          + str(pd.Timestamp.now())[:10]
                                          + ".csv"))
        # Get/Set USERNAME & DESKTOP DIRECTORIES
        self.user_name_dir = os.path.join("C:/Users/", self.user_name)
        self.desktop_dir = os.path.join(self.user_name_dir, self.desktop_dir)
        print(self.user_name_dir)
        print(self.desktop_dir)
        self.run(day_range=30)
    # }
    
    def run(self, day_range): # {
        # TRY THE FOLLOWING
        try: # {
            # [2020-02-28]\\self.time_unit = time_unit
            # [2020-02-28]\\self.time_value = time_value
            self.day_range = day_range
            # get/set current date variable
            # [2020-03-06]\\the_date = pd.Timestamp.now()
            # [2020-03-11]\\the_date = pd.Timestamp(ts_input=str(self.end_date.get()))
            the_date = pd.Timestamp.now()
            # create variable for a month ago
            one_month_ago = the_date - timedelta(days = int(day_range))
            # [2020-02-28]\\
            ##one_month_ago = the_date - pd.Timedelta(unit=str(self.time_unit), value=str(self.time_value))
            print(type(one_month_ago))
            # CREATE METRICS TABLE FROM CLASS METHOD
            self.df_metrics_table = self.create_metrics_table()
            #############################
            # create .csv with no drops #
            #############################
            """
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_metrics_noDrop.csv"), 
                                         index=True)
            # DROP ROWS WITH PRODUCT LEVEL && QCDATE = NONE
            self.df_metrics_table.dropna(axis=0, subset=['QCDate', 'ProductLevel'], how='any', 
                                         inplace=True)
            # create .csv with DROPS
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_metrics_DROP.csv"), 
                                         index=True)
            """
            # Set index of METRICS
            # [2020-03-11]\\self.df_metrics_table.set_index(['QCDate'], inplace=True)
            self.df_metrics_table.index = self.df_metrics_table['QCDate']
            del self.df_metrics_table['QCDate']
            print(self.df_metrics_table)
            # Display Index
            print(self.df_metrics_table.index)
            #############################
            # create .csv with UNSORTED #
            #############################
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_TIS_metrics_UNSORTED.csv"), 
                                         index=True)
            # SORT INDEX
            self.df_metrics_table.sort_index(inplace=True)
            ###########################
            # create .csv with SORTED #
            ###########################
            self.df_metrics_table.to_csv(os.path.join(self.outbound_dir, "df_TIS_metrics_SORTED.csv"), 
                                         index=True)
            print("SORTED:\n")
            print(self.df_metrics_table.index)
            # SLICE & DICE THE DATAFRAME
            # [2020-03-11]\\self.df_level_1s = self.df_metrics_table.loc['2017-01-02']
            print("<<<<< df_level_1s >>>>>\n\t" + str(self.df_metrics_table.loc['2017-01-02':'2020-01-01'].info()))
            # SLICE AND DICE THE DATAFRAME TO A MONTH PERIOD
            df_month = self.df_metrics_table.loc['2020-02-01':'2020-02-29']
            # CREATE EMPTY DATAFRAME TO HOLD THE MONTH OF FEBRUARY
            df_month_FINALE = pd.DataFrame(data=None, dtype=np.str, index=df_month.index)
            # NARROW DOWN ON COLUMNS
            df_month_FINALE["QCDate"] = df_month.index
            df_month_FINALE["OrderID"] = df_month["OrderID"]
            df_month_FINALE["ProductNo"] = df_month["ProductNo"]
            df_month_FINALE["Volume"] = df_month["Volume"]
            df_month_FINALE["BuildByDate"] = df_month["BuildByDate"]
            df_month_FINALE["OrderDate"] = df_month["OrderDate"]
            df_month_FINALE["PrepDate"] = df_month["PrepDate"]
            df_month_FINALE["PrepVolume"] = df_month["PrepVolume"]
            df_month_FINALE["UnitizeNLT"] = df_month["UnitizeNLT"]
            df_month_FINALE["ShipNLTDate"] = df_month["ShipNLTDate"]
            df_month_FINALE["BulkQCDate"] = df_month["BulkQCDate"]
            df_month_FINALE["AmpDate"] = df_month["AmpDate"]
            df_month_FINALE["GreenSheet"] = df_month["GreenSheet"]
            df_month_FINALE["SOLink"] = df_month["SOLink"]
            df_month_FINALE["Notes"] = df_month["Notes"]
            # Export?
            df_month_FINALE.to_csv(os.path.join(self.outbound_dir, "df-TIS-month-"
                                        + str(pd.Timestamp.now())[:10]
                                        + ".csv"), index=True)
            df_merge = pd.merge(df_month_FINALE, self.df_infile, on='ProductNo', how='inner')
            # RESET INDEX BACK TO QCDATE?
            # <<<< >>>> 
            # EXPORT ??
            df_merge.to_csv(os.path.join(self.outbound_dir, "df-TIS-month-MATERIAL-MERGE-"
                                        + str(pd.Timestamp.now())[:10]
                                        + ".csv"), index=True)
            # GET TIMESTMP FOR TODAY
            ts_now = pd.Timestamp.now()
            # create timedelta for LAST WEEK
            td_last_week = pd.Timedelta(days=7)
            # create timedelta for YESTERDAY
            td_yesterday = pd.Timedelta(days=2)
            # create actual TIMESTAMP for last week (whenever it was)
            ts_last_week = pd.Timestamp(ts_now - td_last_week)
            # create actual TIMESTAMP for YESTERDAY
            ts_yesterday = pd.Timestamp(ts_now - td_yesterday)
            ## CREATE DATAFRAME OF LAST WEEK
            df_last_week = self.df_metrics_table.loc[ts_last_week:ts_now]
            ## CREATE DATAFRAME OF YESTERDAY
            df_yesterday = self.df_metrics_table.loc[ts_yesterday:ts_now]
            # EXPORT ??
            df_last_week.to_csv(os.path.join(self.outbound_dir, "df_TIS-last-week-" 
                                             + str(pd.Timestamp.now())[:10] 
                                             + ".csv"))
            print("LAST WEEK COLS:\n" + str(df_last_week.columns))
            # CREATE NEW EMPTY DATAFRAME (to hold SPEICIFIC columns)
            df_yesterday_FINALE = pd.DataFrame(data=None, dtype=np.str, index=df_yesterday.index)
            # [2020-03-12]\\df_yesterday_FINALE["QCDate"] = df_yesterday["QCDate"]
            # [2020-03-24]\\df_yesterday_FINALE["QCDate"] = df_yesterday.index
            df_yesterday_FINALE["QCDate"] = self.df_tblProdflow["QCDate"]
            df_yesterday_FINALE["OrderID"] = df_yesterday["OrderID"]
            df_yesterday_FINALE["ProductNo"] = df_yesterday["ProductNo"]
            df_yesterday_FINALE["Volume"] = df_yesterday["Volume"]
            df_yesterday_FINALE["BuildByDate"] = df_yesterday["BuildByDate"]
            df_yesterday_FINALE["OrderDate"] = df_yesterday["OrderDate"]
            df_yesterday_FINALE["PrepDate"] = df_yesterday["PrepDate"]
            df_yesterday_FINALE["PrepVolume"] = df_yesterday["PrepVolume"]
            # [2020-03-13]\\df_yesterday_FINALE["QCDate"] = df_yesterday["QCDate"]
            # [2020-03-17]\\df_yesterday_FINALE["QCDate"] = df_yesterday.index
            df_yesterday_FINALE["UnitizeNLT"] = df_yesterday["UnitizeNLT"]
            df_yesterday_FINALE["ShipNLTDate"] = df_yesterday["ShipNLTDate"]
            df_yesterday_FINALE["BulkQCDate"] = df_yesterday["BulkQCDate"]
            df_yesterday_FINALE["AmpDate"] = df_yesterday["AmpDate"]
            df_yesterday_FINALE["GreenSheet"] = df_yesterday["GreenSheet"]
            df_yesterday_FINALE["SOLink"] = df_yesterday["SOLink"]
            # [2020-03-17]\\df_yesterday_FINALE["QCExpDate"] = df_yesterday["QCExpDate"]
            # [2020-03-17]\\df_yesterday_FINALE["QCMemo"] = df_yesterday["QCMemo"]
            df_yesterday_FINALE["Notes"] = df_yesterday["Notes"]
            """
            df_yesterday_FINALE["PrepDate"] = df_yesterday["PrepDate"]
            df_yesterday_FINALE["BulkQCDate"] = df_yesterday["BulkQCDate"]
            df_yesterday_FINALE["AmpDate"] = df_yesterday["AmpDate"]
            df_yesterday_FINALE["ProductNo"] = df_yesterday["ProductNo"]
            df_yesterday_FINALE["OrderID"] = df_yesterday["OrderID"]
            df_yesterday_FINALE["OrderDate"] = df_yesterday["OrderDate"]
            df_yesterday_FINALE["Notes"] = df_yesterday["Notes"]
            """
            df_yesterday_FINALE.to_csv(os.path.join(self.outbound_dir, "df_TIS-yesterday-"
                                             + str(pd.Timestamp.now())[:10]
                                             + ".csv"), index=True)
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
        # }
    # }
    
    """
    Referred to as "ProdflowII" in SQL-Server
    """
    def pull_ProdflowII_table(self, table_name): # {
        # TRY THE FOLLOWING
        try: # {
            # CREATION CONNECTION STR
            conn_str = str(
                r'DRIVER={ODBC Driver 17 for SQL Server};'
                r'SERVER=wtkngappflow1.is.agilent.net;'
                r'DATABASE=ProdFlowII_Prod;'
                r'Trusted_Connection=yes;'
            )
            # CREATE PYODBC CONNECTION
            cnxn_ProdflowII = pyodbc.connect(conn_str)
            # [2020-02-028]\\crsr_ProdflowII = cnxn_ProdflowII.cursor()
            # PERFORM SQL QUERY AND SET AS DATAFRAME
            df_ProdflowII_table = pd.read_sql_query(sql='SELECT * FROM ' + str(table_name),
                                                    con=cnxn_ProdflowII
                                                    )
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
            return df_ProdflowII_table
        # }
    # }
    
    """
    Referred to as "Prodflow" in SQL-Server
    """
    def pull_ProdflowIII_table(self, table_name): # {
        # TRY THE FOLLOWING
        try: # {
            # CREATE CONNECTION STRING
            conn_str = str(
                r'DRIVER={ODBC Driver 17 for SQL Server};'
                r'SERVER=wtkngappflow1.is.agilent.net;'
                r'DATABASE=ProdFlow;'
                r'Trusted_Connection=yes;'
            )
            # CREATE PYODBC CONNECTION
            cnxn_ProdflowIII = pyodbc.connect(conn_str)
            # [2020-02-28]\\crsr_ProdflowIII = cnxn_ProdflowIII,cursor()
            # PERFORM SQL QUERY AND SET AS DATAFRAME
            df_ProdflowIII_table = pd.read_sql_query(sql='SELECT * FROM ' + str(table_name),
                                                     con=cnxn_ProdflowIII,
                                                     parse_dates=['QCDate']
                                                     )
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
            return df_ProdflowIII_table
        # }
    # }
    
    def create_metrics_table(self): # {
        # TRY THE FOLLOWING
        try: # {
            # pull ORDERS table
            self.df_orders = self.pull_ProdflowII_table(table_name='Orders')
            # EXPORT A SAMPLE
            self.df_orders.sample(25).to_csv(os.path.join(self.outbound_dir, "df-ORDERS-table-"
                                                         + str(pd.Timestamp.now())[:10]
                                                         + ".csv"))
            # [2020-03-11]\\export_path = os.path.join(self.outbound_dir, "df_orders.csv")
            # [2020-03-11]\\self.df_orders.to_csv(export_path, index=True)
            # RENAME ['Product#'] column to ["ProductNo"]
            self.df_orders.rename(columns={'Product#': 'ProductNo'}, inplace=True)
            print("\tORDERS-TABLE:\n" + str(self.df_orders.info()))
            # pull tblProdflow TABLE
            self.df_tblProdflow = self.pull_ProdflowIII_table(table_name='tblProdflow')
            # EXPORT A SAMPLE
            self.df_tblProdflow.sample(25).to_csv(os.path.join(self.outbound_dir, "df-ORDERS-table-"
                                                         + str(pd.Timestamp.now())[:10]
                                                         + ".csv"))
            print(self.df_tblProdflow.info())
            ########################
            # CREATE METRICS TABLE #
            ########################
            df_metrics_table = pd.merge(self.df_orders, self.df_tblProdflow, on='ProductNo', how='right')
            ######################
            # CHANGE DATA TYPES: #
            ######################
            
            # CHANGE ['QCDate'] COLUMN TO DATETIME
            df_metrics_table['QCDate'] = pd.to_datetime(df_metrics_table['QCDate'])
            ####################################
            # DROP ALL ROWS WITHOUT A 'QCDATE' #
            ####################################
            df_metrics_table.dropna(axis=0, subset=['QCDate'],
                                   how='any', inplace=True)
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n\n" + str(errorMessage) + "\n")
            print("\n" + typeE + 
                  "\n" + fileE + 
                  "\n" + lineE + 
                  "\n" + messageE)
        #}
        else: # {
            print("Operation Completed Successfully...")
            return df_metrics_table
        # }
    # }
    
    def send_email(self, send_from, send_to, subject, message, files=[],
                  server="cos.smtp.agilent.com", port=587, use_tls=True): # {
        print("SENDING E-MAIL...\nDATE == " + str(pd.Timestamp.now())[:10])
        msg = MIMEMultipart()
        msg['From'] = send_from
        msg['To'] = COMMASPACE.join(send_to)
        msg['Date'] = formatdate(localtime=True)
        msg['Subject'] = subject
        
        msg.attach(MIMEText(message))
    # }
    
# }

In [3]:
if __name__ == "__main__": # {
    test_metrics = TISS_Metrics(the_logger=None, 
                                materials_file="C:/data/inbound/2020-03-19/material_list.csv")
# }

C:/Users/derbates
C:/Users/derbates\OneDrive - Agilent Technologies/Desktop
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Operation Completed Successfully...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106559 entries, 0 to 106558
Data columns (total 28 columns):
OrderID            106559 non-null int64
Quote#             106553 non-null object
ProductNo          106559 non-null object
LineID             106550 non-null float64
Quantity           106490 non-null float64
SaleUnit           95030 non-null object
Company            106293 non-null object
Volume             106293 non-null float64
VolUnit            40636 non-null object
OrderDate          106553 non-null datetime64[ns]
BuildByDate        106008 non-null object
ShipNLTDate        105974 non-null object
Route              106558 non-null float64
FullValidation     106559 non-null bool
Validation         76105 non-null float64
Built              106559 non-null bool
GreenSheet         78364 non-null object
Notes   

SORTED:

DatetimeIndex(['1987-01-01', '1987-01-01', '1989-01-01', '1991-01-01',
               '1991-01-01', '1991-01-01', '1991-01-01', '1992-01-01',
               '1992-01-01', '1992-01-01',
               ...
               '2020-03-25', '2020-03-25', '2020-03-25', '2020-03-25',
               '2020-03-25', '2020-03-25', '2020-03-25', '2020-03-25',
               '2020-03-25', '2020-03-25'],
              dtype='datetime64[ns]', name='QCDate', length=1800304, freq=None)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 488341 entries, 2017-01-03 to 2019-12-31
Data columns (total 79 columns):
OrderID             487197 non-null float64
Quote#              487193 non-null object
ProductNo           488341 non-null object
LineID              487194 non-null float64
Quantity            487188 non-null float64
SaleUnit            463217 non-null object
Company             486733 non-null object
Volume              486622 non-null float64
VolUnit             110728 non-null object
Ord